<a href="https://colab.research.google.com/github/fdagreat/Hybrid_Demucs_V3_vs_V4_Music_Source_Separation_Comparison/blob/main/Hybrid_Demucs_V3_vs_V4_Music_Source_Separation_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.


v3_path = '/gdrive/MyDrive/demucs_separated/mdx_extra_q'
v4_path = '/gdrive/MyDrive/demucs_separated/htdemucs'

In [ ]:
import os
import shutil
import matlab.engine
from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [ ]:
import os
import shutil
import matlab.engine

# Specify the directories containing the audio files
directory1 = str(v3_path)
directory2 = str(v4_path)

# Create a temporary directory to store the audio files
temp_directory = '/path/to/temp_directory'
os.makedirs(temp_directory, exist_ok=True)

# Get a list of files in the first directory
files1 = os.listdir(directory1)

# Copy the corresponding audio files to the temporary directory
for file in files1:
    source_file = os.path.join(directory1, file)
    destination_file = os.path.join(temp_directory, file)
    shutil.copyfile(source_file, destination_file)

# Start the MATLAB engine
matlab_engine = matlab.engine.start_matlab()

# Perform wavelength comparison for each audio file
for file in files1:
    audio_file1 = os.path.join(directory1, file)
    audio_file2 = os.path.join(directory2, file)
    temp_file = os.path.join(temp_directory, file)

    # Load the audio files in MATLAB
    matlab_engine.eval(f"audio1 = audioread('{audio_file1}');")
    matlab_engine.eval(f"audio2 = audioread('{audio_file2}');")
    matlab_engine.eval(f"temp_audio = audioread('{temp_file}');")

    # Perform wavelength comparison using a MATLAB function
    matlab_engine.eval("result1 = compare_wavelength(audio1, temp_audio);")
    matlab_engine.eval("result2 = compare_wavelength(audio2, temp_audio);")

    # Retrieve the results from MATLAB
    result1 = matlab_engine.workspace['result1']
    result2 = matlab_engine.workspace['result2']

    print(f"Wavelength comparison for {file}:")
    print(f"Result 1: {result1}")
    print(f"Result 2: {result2}")
    print()

# Close the MATLAB engine
matlab_engine.quit()

# Remove the temporary directory and its contents
shutil.rmtree(temp_directory)
